# List Active Seasons for Each Housewife By Series

## Import Libraries

In [9]:
import pandas as pd
from urllib.request import urlopen
from bs4 import BeautifulSoup
import re
url_list = pd.read_csv('./data/series_urls.csv')

## Print Housewives by Series

In [ ]:
# Go through the list of URLs for each series.
for index, row in url_list.iterrows():
    series = row['Series']
    soup = BeautifulSoup(urlopen(row['URL']))
    table = soup.find('table', class_='wikitable')
    rows = table.find_all('tr')

    # Loop through each row containing the housewife's name and their active seasons.
    for row in rows[2:]:
        # Ignore Friends of the Housewives since they're not the main cast.
        if row.text.strip().lower() == 'friends of the housewives':
            break

        # The table data in the row contains the housewife's name and their role in a particular season.
        tds = row.find_all('td')
        name = tds[0].text.strip()
        name_parts = name.split(' ', 1)
        first_name = name_parts[0].replace("'", "''")
        last_name = name_parts[1].replace("'", "''")

        # Elizabeth Lyn Vargas is the only housewife whose credited middle name is not part of a double surname.
        if last_name == 'Lyn Vargas':
            first_name = 'Elizabeth Lyn'
            last_name = 'Vargas'

        count = 1

        for td in tds[1:]:
            role = td.text.strip()
            columns = td.get('colspan')
            columns = re.findall(r'\d+', columns)[0] # Removes non-number characters
            columns = int(columns)

            # Add to count for season that housewife did not appear in main role.
            if role == '' or role == 'Guest' or role == 'Friend':
                count += columns
                continue
            
            # Print out the seasons where the housewife had a main role.
            for i in range(columns):
                print(f"""(
    (
        SELECT
            season_id
        FROM
            seasons
        WHERE
            series_id = '{series}'
            AND season_number = {count + i}
    ),
    (
        SELECT
            housewife_id
        FROM
            housewives
        WHERE
            first_name = '{first_name}'
            AND last_name = '{last_name}'
    )
),""")
            count += columns

            # print(f"Active Seasons: {count - 1}")

(
    (
        SELECT
            season_id
        FROM
            seasons
        WHERE
            series_id = 'RHOC'
            AND season_number = 1
    ),
    (
        SELECT
            housewife_id
        FROM
            housewives
        WHERE
            first_name = 'Kimberly'
            AND last_name = 'Bryant'
    )
),
(
    (
        SELECT
            season_id
        FROM
            seasons
        WHERE
            series_id = 'RHOC'
            AND season_number = 1
    ),
    (
        SELECT
            housewife_id
        FROM
            housewives
        WHERE
            first_name = 'Jo'
            AND last_name = 'De La Rosa'
    )
),
(
    (
        SELECT
            season_id
        FROM
            seasons
        WHERE
            series_id = 'RHOC'
            AND season_number = 2
    ),
    (
        SELECT
            housewife_id
        FROM
            housewives
        WHERE
            first_name = 'Jo'
            AND last_name = 